## Custom loss functions

### 1. Weighted
Gives *more* weight to under-represented class and *less* weight to over-represented class.

$$loss_{+}^{(i)} = -1 \times w_{+}^{(i)} \times y^{(i)} \times log(\hat{y}^{(i)})$$
$$loss_{-}^{(i)} = -1 \times w_{-}^{(i)} \times (1- y^{(i)}) \times log(1 - \hat{y}^{(i)})$$

$$ loss^{(i)} = loss_{+}^{(i)} + los_{-}^{(i)} $$

In [ ]:
import numpy as np

y_true = np.array(
        [[1,0],
         [1,0],
         [1,0],
         [1,0],
         [0,1]
        ])

y_pred = np.ones(y_true.shape)
y_pred[:,0] = 0.3 * y_pred[:,0]
y_pred[:,1] = 0.7 * y_pred[:,1]
y_pred

array([[0.3, 0.7],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.3, 0.7]])

In [ ]:
w_p = np.sum(y_true == 0,axis=0) / y_true.shape[0]
w_n = np.sum(y_true == 1, axis=0) / y_true.shape[0]

w_p, w_n

(array([0.2, 0.8]), array([0.8, 0.2]))

In [ ]:
loss_0_pos = -1 * np.sum(w_p[0] * 
                y_true[:, 0] * 
                np.log(y_pred[:, 0])
              )

loss_0_neg = -1 * np.sum( 
                w_n[0] * 
                (1 - y_true[:, 0]) * 
                np.log(1 - y_pred[:, 0])
              )

loss_0 = loss_0_neg + loss_0_pos
loss_0

1.2485181986117349

In [ ]:
loss_1_pos = -1 * np.sum(w_p[1] * 
                         y_true[:, 1] * 
                         np.log(y_pred[:, 1])
                        )

loss_1_neg = -1 * np.sum(w_n[1] * 
                         (1 - y_true[:, 1]) * 
                         (np.log(1 - y_pred[:, 1])) 
                        )

loss_1 = loss_1_pos + loss_1_neg
loss_1

1.2485181986117349

### 2. Soft Dice coefficient

The soft dice coefficient loss function takes real input, instead of discrete numbers, which measures how well two images (here, the model's prediction and ground truth) overlap.

$$\mathcal{L}_{Dice}(p, q) = 1 - \frac{1}{N} \sum_{c=1}^{C} \frac{2\times\sum_{i, j} p_{cij}q_{cij} + \epsilon}{\left(\sum_{i, j} p_{cij}^2 \right) + \left(\sum_{i, j} q_{cij}^2 \right) + \epsilon}$$

In [ ]:
from keras.backend import sum, mean

def soft_dice_loss(y_pred, y_true, axis=(1, 2, 3), eps=0.00001):
    num = sum(y_pred * y_true, axis)
    den = sum(y_pred**2, axis) + sum(y_true**2, axis)
    coeff = 1 - mean((2 * num + eps) / (den + eps))

    return coeff